In [2]:
import os
share4v_balanced="/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/robustlmm/sharegpt4v/new/balanced/pt_bal1174k_ft_orig"

file_list=["/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/robustlmm/dr_algo/supply_benchmarks/qbench_share4v_pt_tocp0a1_ft_orig/0831_1728_qbenc..._orig_sharegpt4v_model_args_00bfe7"]
for benchmark_dir in os.listdir(share4v_balanced):
    benchmark_dir_path = os.path.join(share4v_balanced, benchmark_dir)
    for sub_benchmark_dir in os.listdir(benchmark_dir_path):
        if os.path.isdir(os.path.join(benchmark_dir_path, sub_benchmark_dir)):
            sub_benchmark_dir_path = os.path.join(benchmark_dir_path, sub_benchmark_dir)
            if "results.json" in os.listdir(sub_benchmark_dir_path):
                file_list.append(os.path.join(sub_benchmark_dir_path, "results.json"))
        
        
        

In [3]:
import os
from mhr.utils.utils import load_json_file,process_jsonl


def read_results_from_file(file_path):
    if not isinstance(file_path,list):
        file_path = [file_path]
    result_dict = {}
    for path in file_path:
        if os.path.isdir(path):
            path = os.path.join(path, "results.json")
        result_dict.update(load_json_file(path)["results"])
        # result_dict.append(load_json_file(path)["results"])
    return result_dict

share4v_base_file=[
    "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/robustlmm/sharegpt4v/0723_0003_share4v_baseline_sharegpt4v_model_args_f37f0e",
    "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/robustlmm/dr_algo/supply_benchmarks/qbench_share4vbench/0825_1838_qbenc...bench_sharegpt4v_model_args_f37f0e"
]
share4v_balanced_file=file_list
share4v_aug_file="/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/robustlmm/sharegpt4v/new/aug/ptall1168k_diff_ftorig/0902_0032_ptall...torig_sharegpt4v_model_args_964e32"

llava_base_file=[
    "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/robustlmm/dr_algo/0605_2135_eval_all_llava_baseline_llava_model_args_ca10f3",
    "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/robustlmm/dr_algo/supply_benchmarks/qbench_llavabase/0827_1159_qbenc...abase_llava_model_args_ca10f3",
    
]
llava_balanced_file = [
    "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/robustlmm/dr_algo/0704_2150_eval_all_dr_algo_ptn_ft0_new_llava_model_args_be754a",
    "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/robustlmm/dr_algo/supply_benchmarks/qbench_llavaallp0a1/0827_1207_qbenc...lp0a1_llava_model_args_be754a",
]

llava_aug_file = [
    "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/robustlmm/dr_algo/0721_1616_all_a1_aug_665k_llava_model_args_cd7cf0",
    "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/robustlmm/dr_algo/supply_benchmarks/qbench_llavaallp0a1diff/0827_1214_qbenc...1diff_llava_model_args_73fa2a",
]

llava_base_data = read_results_from_file(llava_base_file)
llava_balanced_data = read_results_from_file(llava_balanced_file)
llava_aug_data = read_results_from_file(llava_aug_file)

share4v_base_data = read_results_from_file(share4v_base_file)
share4v_balanced_data = read_results_from_file(share4v_balanced_file)
share4v_aug_data = read_results_from_file(share4v_aug_file)



In [4]:

metric_dict = {
     "mmstar": ["coarse perception,none","fine-grained perception,none","instance reasoning,none","logical reasoning,none","science & technology,none","math,none","average,none"],
     "pope": ["pope_precision,none","pope_recall,none","pope_f1_score,none","pope_accuracy,none",],
     "flickr30k": ["flickr_Bleu_4,none","flickr_METEOR,none","flickr_ROUGE_L,none","flickr_CIDEr,none"],
     "seedbench_2": ["seed_Single_Image,none","seed_Multiple_Images,none","seed_Interleaved_Image,none","seed_Image_Generation,none","seed_Image_&_Text_Generation,none","seed_Video,none","seed_all,none"],
}
metric1_list=["flickr30k","seedbench_2"]
metric2_list=["pope","mmstar",]




In [16]:


method_dict = {
    "ShareGPT4V":{"data":share4v_base_data,"ft_length":f"{1246 :.1f}K",},
    "\hspace{0.3cm}\\textcolor{blue}{+balance}":{"data":share4v_balanced_data,"ft_length":f"{1168 :.1f}K",},
    "\hspace{0.3cm}\\textcolor{red}{+augment}":{"data":share4v_aug_data,"ft_length":f"{1246 :.1f}K",},
}

method_list = list(method_dict.keys())

method2_dict = {
    "LLaVA 1.5":{"data":llava_base_data,"ft_length":f"{665 :.1f}K",},
    "\hspace{0.3cm}\\textcolor{blue}{+balance}":{"data":llava_balanced_data,"ft_length":f"{581 :.1f}K",},
    "\hspace{0.3cm}\\textcolor{red}{+augment}":{"data":llava_aug_data,"ft_length":f"{665 :.1f}K",},
}
method2_list = list(method2_dict.keys())

In [17]:
def get_results_to_list(method_list,method_dict,metric_list,metric_dict):
    res = {}
    for method in method_list:
        data = method_dict[method]["data"]
        method_res = []
        for metric in metric_list:
            metric_avg_acc = []
            for metric_idx,metric_item in enumerate(metric_dict[metric]):
                if metric == "seedbench_2":
                    metric_res = data.get(metric,data.get("seedbench-2",None)).get(metric_item,-1)
                else:
                    metric_res = data[metric].get(metric_item,-1)
                if metric not in ["mme","mmbench_cn_dev","mmbench_en_dev","llava_bench_coco","llava_in_the_wild"]:
                    metric_res *= 100
                if metric_item == "average,none" and metric_res == -100:
                    metric_res = sum(metric_avg_acc)/len(metric_avg_acc)
                
                metric_avg_acc.append(metric_res)
                method_res.append(metric_res)
        res[method] = method_res
    return res

def form_latex_str(res,method_dict):
    res_str = ""
    method_list = list(res.keys())
    res_dict = {method:f"{method} & {method_dict[method]['ft_length']} " for method in method_list}
    avg_dict = {method:[] for method in method_list}
    for i in range(len(res[method_list[0]])):
        compare_list = sorted([res[method][i] for method in method_list])
        for method in method_list:
            if res[method][i] == compare_list[-1] and (i == len(res[method_list[0]])-1 or i==3):
                res_dict[method] += f"& \\textbf{{{res[method][i]:.1f}}}"
            # elif res[method][i] == compare_list[-2]:
            #     res_dict[method] += f"& \\underline{{{res[method][i]:.1f}}}"
            else:
                res_dict[method] += f"& {res[method][i]:.1f}"
            avg_dict[method].append(res[method][i])
    avg_dict = {method:f" & {sum(avg_dict[method])/len(avg_dict[method]):.1f}" for method in method_list}
    for method in method_list:
        res_str += res_dict[method] + "\\\\ \n"
    return res_str
        
def form_excel_str(res,method_dict):
    res_str = ""
    method_list = list(res.keys())
    res_dict = {method:f"{method}\t{method_dict[method]['pt_length']}\t{method_dict[method]['ft_length']}" for method in method_list}
    avg_dict = {method:[] for method in method_list}
    for i in range(len(res[method_list[0]])):
        compare_list = sorted([res[method][i] for method in method_list])
        for method in method_list:
            
            res_dict[method] += f"\t{res[method][i]:.1f}"
            avg_dict[method].append(res[method][i])
    avg_dict = {method:f"\t{sum(avg_dict[method])/len(avg_dict[method]):.1f}" for method in method_list}
    for method in method_list:
        res_str += res_dict[method] + "\n"
    return res_str

In [18]:
# metric1
res = get_results_to_list(method2_list,method2_dict,metric1_list,metric_dict)
res_str = form_latex_str(res,method2_dict)

res2 = get_results_to_list(method_list,method_dict,metric1_list,metric_dict)
res_str2 = form_latex_str(res2,method_dict)
res_str = res_str + "\hline\n" + res_str2
print(res_str)

LLaVA 1.5 & 665.0K & 28.6& 26.2& 52.9& 74.9& 65.4& 51.2& 38.5& 24.5& 93.7& 42.1& \textbf{58.0}\\ 
\hspace{0.3cm}\textcolor{blue}{+balance} & 581.0K & 29.5& 26.3& 53.4& \textbf{76.2}& 65.4& 47.6& 42.0& 0.0& 89.9& 43.9& 57.2\\ 
\hspace{0.3cm}\textcolor{red}{+augment} & 665.0K & 29.3& 26.5& 53.5& 76.2& 66.0& 48.5& 40.8& 0.0& 93.7& 42.7& 57.4\\ 
\hline
ShareGPT4V & 1246.0K & 30.5& 26.9& 54.0& 78.5& 67.3& 55.2& 32.5& 25.0& 89.9& 43.1& 59.6\\ 
\hspace{0.3cm}\textcolor{blue}{+balance} & 1168.0K & 30.6& 26.8& 54.1& \textbf{78.9}& 67.5& 54.1& 37.3& 24.9& 92.4& 42.3& 59.6\\ 
\hspace{0.3cm}\textcolor{red}{+augment} & 1246.0K & 30.4& 26.9& 54.0& 78.8& 67.7& 53.5& 34.3& 25.0& 92.4& 42.6& \textbf{59.8}\\ 



In [19]:
# metric2
res = get_results_to_list(method2_list,method2_dict,metric2_list,metric_dict)
res_str = form_latex_str(res,method2_dict)

res2 = get_results_to_list(method_list,method_dict,metric2_list,metric_dict)
res_str2 = form_latex_str(res2,method_dict)
res_str = res_str + "\hline\n" + res_str2
print(res_str)

LLaVA 1.5 & 665.0K & 94.1& 78.7& 85.7& 86.9& 63.9& 24.7& 39.3& 28.6& 17.2& 27.3& 33.5\\ 
\hspace{0.3cm}\textcolor{blue}{+balance} & 581.0K & 94.1& 79.4& 86.1& \textbf{87.2}& 61.1& 29.4& 40.2& 31.2& 19.1& 21.9& 33.8\\ 
\hspace{0.3cm}\textcolor{red}{+augment} & 665.0K & 94.1& 78.7& 85.7& 86.9& 64.7& 28.3& 41.7& 32.1& 17.9& 28.2& \textbf{35.5}\\ 
\hline
ShareGPT4V & 1246.0K & 94.5& 78.0& 85.5& 86.8& 67.8& 26.3& 44.2& 28.7& 15.1& 25.9& 34.7\\ 
\hspace{0.3cm}\textcolor{blue}{+balance} & 1168.0K & 93.8& 79.3& 85.9& \textbf{87.0}& 66.3& 29.5& 44.0& 28.0& 16.8& 23.7& 34.7\\ 
\hspace{0.3cm}\textcolor{red}{+augment} & 1246.0K & 93.7& 79.3& 85.9& 87.0& 66.1& 30.7& 43.9& 30.4& 17.2& 24.9& \textbf{35.5}\\ 



## ShareGPT4V

In [40]:
res = get_results_to_list(method_list,method_dict,metric1_list,metric_dict)
res_str = form_latex_str(res,method_dict)
print(res_str)

ShareGPT4V & 1246.0K & 30.5& \textbf{26.9}& 54.0& 78.5& 67.3& \textbf{55.2}& 32.5& \textbf{25.0}& 89.9& \textbf{43.1}& 59.6\\ 
Balanced ShareGPT4V & 1168.0K & \textbf{30.6}& 26.8& \textbf{54.1}& \textbf{78.9}& 67.5& 54.1& \textbf{37.3}& 24.9& \textbf{92.4}& 42.3& 59.6\\ 
Ours & 1246.0K & 30.4& 26.9& 54.0& 78.8& \textbf{67.7}& 53.5& 34.3& \textbf{25.0}& \textbf{92.4}& 42.6& \textbf{59.8}\\ 



In [41]:
res = get_results_to_list(method_list,method_dict,metric2_list,metric_dict)
res_str = form_latex_str(res,method_dict)
print(res_str)

ShareGPT4V & 1246.0K & \textbf{94.5}& 78.0& 85.5& 86.8& \textbf{67.8}& 26.3& \textbf{44.2}& 28.7& 15.1& \textbf{25.9}& 34.7\\ 
Balanced ShareGPT4V & 1168.0K & 93.8& 79.3& \textbf{85.9}& \textbf{87.0}& 66.3& 29.5& 44.0& 28.0& 16.8& 23.7& 34.7\\ 
Ours & 1246.0K & 93.7& \textbf{79.3}& 85.9& 87.0& 66.1& \textbf{30.7}& 43.9& \textbf{30.4}& \textbf{17.2}& 24.9& \textbf{35.5}\\ 

